In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from src.definitions import ROOT_DIR
from src.data.utils import download_from_groningen, head, line_count
from src.data.quality import check_grid_spacing
from src.visualization.visualize import (
    index_depth,
    get_meshes_from_gridded_surface_pointset,
    plot_cartesian_gridded_surface
)

# The Groningen field

In this section, we present some of the relevant and interesting facts of the Groningen gas field, extracted from the field's geologic overview by de Jager and Visser (2017). The Groningen field data is vast, but we can use this information to find the files needed for our study. 

The Groningen gas field was discovered in 1959 by the Slochteren-1 well. This well primary target was to test the gas trend in the Basal Zechstein carbonates, which resulted in tight basinal facies. However, the underlying Rotliegend sandstones unexpectedly found a large volume of gas. The initial recoverable gas reserves are estimated at 2900 bcm (~100 TCF). Additional field information is summarized in table 1.

Table 1: Groningen field summary.
|               | Description |
| ------------- | ----------- |
| **Structure** | Faulted anticline (4-way). The closure formed during the late Triassic or Jurassic. The primary faults have normal displacement and trend NNW–SSE. Secondary fault trends run E-W and N-S. |
| **Seal** | The top seal consists of carbonates and anhydrites layers from the Zechstein Formation. Laterally, the gas accumulation is mainly constrained by faults. |
| **Source** | Main source rocks (Carboniferous age) are the underlying and juxtaposed Westphalian Coal Measures and the basal Namurian organic shales. The primary expulsion event occurred during the Late Jurassic - Early Cretaceous. |
| **Reservoir** | Upper Permian Rotliegend reworked aeolian and fluvial sediments. High degree of sorting. Porosity ranges from 10-24 %, and permiability from 1-1000 mD. Three facies: Sandstones, conglomerates, and mudstones. |
| **FWL** | Changes by fault block, from 2972-3016 m TVNAP (NAM, 2016).|


# The Groningen data

In 2020, Nederlandse Aardolie Maatschappij (NAM) released the Groningen gas field geological model throught the Utrecht University data publication platform under a CC BY 4.0 license. These data consists of well logs, seismic horizons, a PSDM seismic cube, and the field's geocellular model, all embedded in a Petrel project. In the same year, Data Underground (2020) forked NAM's project, and made most of its data accessible in common sharing formats (CSV, LAS, SEGY, etc.). In this work, we access the data in the Data Underground repository.

The first step in our workflow is to download the data that we will use to recreate the seismic uncertainty analysis (SUA) surfaces. The typical result of a SUA is a set of P10-P50-P90 values that describe the in-place rock volume distribution of a reservoir, based on multiple iterations of the target's structural top which are derived from equiprobable imaging velocity models.

In our case, we are interested in creating multiple iterations of the reservoir structural top, so we should be looking for horizons and well tops that correspond to the Rotliegend, in addition to any other information (images, reports, README files) that will help us understand the data. 


# Data download

The data files are hosted on an Amazon's S3 bucket and can be downloaded by requesting a URL with the base repository location plus a given file name. The base repository location is

`https://swung-hosted.s3.ca-central-1.amazonaws.com/`

By reading the [Data Underground](https://dataunderground.org/dataset/groningen-open-fork/resource/20fb48a1-5ed4-4a11-8f6a-ff0a7746c58a?inner_span=True) fork we find that there is a file named `groningen/FILENAMES.txt` that lists the names of the files stored on the data repository. We can use this file name list to select the files to download.

For convenience, we added the function `download_from_groningen`, that does the URL building and requesting for us. Let's use it to dowload `groningen/FILENAMES.txt`

In [ ]:
# List with the file names to download
files_to_download = ["groningen/FILENAMES.txt"]

# Local place to save the downloaded files
dst_dir = ROOT_DIR / "data/external/"

In [ ]:
download_from_groningen(files_to_download, dst_dir, overwrite=False)

If all ran without errors, there should be a new file under `<project_dir>/groningen/FILENAMES.txt`.

We can open this text file to explore all of its content, looking for data useful to our study, that is, the top reservoir surface, well tops, and any pictures.

In [ ]:
# List with the file names to download
files_to_download = [
    "groningen/README.txt",
    "groningen/FILENAMES.txt",
    "groningen/Horizon_Interpretation/RO____T",
    "groningen/Horizon_Interpretation/RO____T.crsmeta.xml",
    "groningen/Formation_tops/Groningen__Formation_tops__EPSG_28992.csv",
    "groningen/3DGrid/3D_Grid_Export_settings.PNG",
    "groningen/3DGrid/3D_Grid_Horizon_order.png",
]

In [ ]:
download_from_groningen(files_to_download, dst_dir, overwrite=False)

![Static model horizon order](../data/external/groningen/3DGrid/3D_Grid_Horizon_order.png)

# Top of Top Rotliegend (Permian)

In [ ]:
interp_RO_T_path = dst_dir / files_to_download[2]

line_count(interp_RO_T_path)

In [ ]:
head(interp_RO_T_path)

In [ ]:
col_names = ["inline", "xline", "easting", "northing", "depth"]
ro_t = pd.read_csv(interp_RO_T_path, sep=r"\s+", header=None, names=col_names)

In [ ]:
ro_t.info()

In [ ]:
unique_northing = check_grid_spacing(ro_t.northing)

In [ ]:
unique_northing_diff_mask = np.diff(unique_northing) != 50

unique_northing_off_values =  unique_northing[:-1][unique_northing_diff_mask]

In [ ]:
unique_easting = check_grid_spacing(ro_t.easting)

In [ ]:
unique_easting_diff_mask = np.diff(unique_easting) != 50
unique_easting_off_values =  unique_easting[:-1][unique_easting_diff_mask]

In [ ]:
vmin = ro_t.northing.min()
vmax = ro_t.northing.max()
xmin = ro_t.easting.min()
xmax = ro_t.easting.max()

In [ ]:
title = "Top Rotliegend"
fig, ax = plot_cartesian_gridded_surface(ro_t, title=title)
ax.hlines(unique_northing_off_values,xmin=xmin, xmax=xmax)
ax.vlines(unique_easting_off_values, ymin=vmin, ymax=vmax)
plt.show()

<img src="https://static.cambridge.org/binary/version/id/urn:cambridge.org:id:binary:20180220025359001-0895:S0016774617000245:S0016774617000245_fig1g.jpeg"  width="40%" height="40%">

In [ ]:
title = "Top Rotliegend"
plot_cartesian_gridded_surface(ro_t, title=title)
plt.show()